# ТЕСТОВОЕ ЗАДАНИЕ НА ВАКАНСИЮ “Backend разработчик (Python, Junior)”

In [1]:
import requests as rq
import pandas as pd

In [3]:
path_to_data = "https://file.notion.so/f/s/0f8850ad-e46f-4f37-99ea-0e4e2a6af5b6/trial_task.json?id=2583a04b-4256-4c1f-939e-6eac0f749ceb&table=block&spaceId=41165294-a784-489a-a401-1a916d020564&expirationTimestamp=1689868800000&signature=-1D4yL8SjErQtHRmRLR8SDvsSk6n_DPQ5uefnI1oeLs&downloadName=trial_task.json"
response = rq.get(path_to_data)
response.status_code

200

In [6]:
data = response.json()
data

[{'order_id': 11973,
  'warehouse_name': 'Мордор',
  'highway_cost': -70,
  'products': [{'product': 'ломтик июльского неба',
    'price': 450,
    'quantity': 1},
   {'product': 'билет в Израиль', 'price': 1000, 'quantity': 3},
   {'product': 'статуэтка Ленина', 'price': 200, 'quantity': 3}]},
 {'order_id': 62239,
  'warehouse_name': 'хутор близ Диканьки',
  'highway_cost': -15,
  'products': [{'product': 'билет в Израиль', 'price': 1000, 'quantity': 1}]},
 {'order_id': 85794,
  'warehouse_name': 'отель Лето',
  'highway_cost': -50,
  'products': [{'product': 'зеленая пластинка', 'price': 10, 'quantity': 2}]},
 {'order_id': 33684,
  'warehouse_name': 'Мордор',
  'highway_cost': -30,
  'products': [{'product': 'билет в Израиль', 'price': 1000, 'quantity': 2},
   {'product': 'зеленая пластинка', 'price': 10, 'quantity': 1}]},
 {'order_id': 25824,
  'warehouse_name': 'отель Лето',
  'highway_cost': -75,
  'products': [{'product': 'автограф Стаса Барецкого',
    'price': 600,
    'quantit

In [9]:
df = pd.read_json(path_to_data)
df

,order_id,warehouse_name,highway_cost,products
0,11973,Мордор,-70,"[{'product': 'ломтик июльского неба', 'price':..."
1,62239,хутор близ Диканьки,-15,"[{'product': 'билет в Израиль', 'price': 1000,..."
2,85794,отель Лето,-50,"[{'product': 'зеленая пластинка', 'price': 10,..."
3,33684,Мордор,-30,"[{'product': 'билет в Израиль', 'price': 1000,..."
4,25824,отель Лето,-75,"[{'product': 'автограф Стаса Барецкого', 'pric..."
...,...,...,...,...
95,28106,хутор близ Диканьки,-30,"[{'product': 'подписка на suppi-блог', 'price'..."
96,98100,остров невезения,-10,"[{'product': 'автограф Стаса Барецкого', 'pric..."
97,79293,отель Лето,-75,"[{'product': 'статуэтка Ленина', 'price': 200,..."
98,2930,Мордор,-30,"[{'product': 'плюмбус', 'price': 250, 'quantit..."


In [12]:
df = pd.json_normalize(data, 'products', ['order_id', 'warehouse_name', 'highway_cost'])
df

,product,price,quantity,order_id,warehouse_name,highway_cost
0,ломтик июльского неба,450,1,11973,Мордор,-70
1,билет в Израиль,1000,3,11973,Мордор,-70
2,статуэтка Ленина,200,3,11973,Мордор,-70
3,билет в Израиль,1000,1,62239,хутор близ Диканьки,-15
4,зеленая пластинка,10,2,85794,отель Лето,-50
...,...,...,...,...,...,...
192,автограф Стаса Барецкого,600,1,79293,отель Лето,-75
193,ломтик июльского неба,450,1,79293,отель Лето,-75
194,плюмбус,250,2,2930,Мордор,-30
195,плюмбус,250,1,2930,Мордор,-30
